In [5]:
from ortools.linear_solver import pywraplp
import openpyxl
import os
import sys
import time
import datetime
from datetime import datetime as dt

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import pandas as pd

In [6]:
solver = pywraplp.Solver.CreateSolver('GLOP')
solver.EnableOutput()
# solver.SetNumThreads(10)

In [7]:
path = './data/'

In [8]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

DEM = pd.read_excel(path+'dataLarge.xlsx',sheet_name='DEM')
CAPA = pd.read_excel(path+'dataLarge.xlsx',sheet_name='CAPA')
PCOST = pd.read_excel(path+'dataLarge.xlsx',sheet_name='PCOST')
SCOST = pd.read_excel(path+'dataLarge.xlsx',sheet_name='SCOST')
ISTOCK = pd.read_excel(path+'dataLarge.xlsx',sheet_name='ISTOCK')
ITEM = DEM['ITEM'].drop_duplicates().to_list()
RES = CAPA['RES'].drop_duplicates().to_list()
TIME = DEM['TIME'].drop_duplicates().to_list()

arr_DEM = DEM.to_numpy()
arr_CAPA = CAPA.to_numpy()
arr_PCOST = PCOST.to_numpy()
arr_SCOST = SCOST.to_numpy()
arr_ISTOCK = ISTOCK.to_numpy()

adj_DEM = np.zeros((len(ITEM),len(TIME)))
adj_CAPA = np.zeros((len(RES),len(TIME)))
adj_PCOST = np.zeros((len(ITEM),len(RES)))
adj_SCOST = np.zeros((len(ITEM)))
adj_ISTOCK = np.zeros((len(ITEM)))

for i in range(len(arr_DEM)):
    demand = arr_DEM[i][2]
    np.put(adj_DEM,i,demand)
    
for i in range(len(arr_CAPA)):
    capacity = arr_CAPA[i][2]
    np.put(adj_CAPA,i,capacity)
    
for i in range(len(arr_PCOST)):
    productcost = arr_PCOST[i][2]
    np.put(adj_PCOST,i,productcost)
    
for i in range(len(arr_SCOST)):
    stockcost = arr_SCOST[i][1]
    np.put(adj_SCOST,i,stockcost)
    
for i in range(len(arr_ISTOCK)):
    initstock = arr_ISTOCK[i][1]
    np.put(adj_ISTOCK,i,initstock)
    
endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Load Data :" + str(endDateTime-startDateTime))

Start : 2023-Feb-03 15:58:20.051687
End : 2023-Feb-03 15:58:20.729441
Load Data :0:00:00.677754


In [9]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

prodqty = {}
storeqty = {}

for i in range(len(ITEM)):
    prodqty[i]=[[solver.NumVar(0, solver.infinity(), 'prodqty[%i][%i][%i]' %(i,j,k)) for k in range(len(TIME))] for j in range(len(RES))]

for i in range(len(ITEM)):
    storeqty[i] = [solver.NumVar(0, solver.infinity(), 'storeqty[%i][%i]' %(i,k)) for k in range(len(TIME))]

prodqty_cnt = len(ITEM)*len(RES)*len(TIME)
storeqty_cnt = len(ITEM)*len(TIME)
   
print('Number of variables =', solver.NumVariables())

Start : 2023-Feb-03 15:58:20.734403
Number of variables = 505000


### demand constraint

In [10]:
for i in range(len(ITEM)) :
    for k in range(len(TIME)):
        if k == 0 :
            solver.Add(solver.Sum(prodqty[i][j][k] for j in range(len(RES))) == int(adj_DEM[i][k]) - int(adj_ISTOCK[i]))
        else :
            solver.Add(solver.Sum(prodqty[i][j][k] for j in range(len(RES))) + storeqty[i][k-1] - storeqty[i][k] == int(adj_DEM[i][k]))

### capa

In [11]:
for j in range(len(RES)):
    for k in range(len(TIME)):
        solver.Add(solver.Sum(prodqty[i][j][k] for i in range(len(ITEM))) <= int(adj_CAPA[j][k]) )

In [12]:
print('Number of constraints =', solver.NumConstraints())

endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Make Constraints:" + str(endDateTime-startDateTime))

Number of constraints = 10000
End : 2023-Feb-03 15:58:22.963230
Make Constraints:0:00:02.228827


### Objective

In [13]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

obj_exp = 0
# obj = solver.Sum(((prodqty[i][j][k] * PCOST[(PCOST['ITEM']==i+1)&(PCOST['RES']==j+1)].reset_index()['PCOST'] for i in range(len(ITEM))) for j in range(len(RES))) for k in range(len(TIME)))
for i in range(len(ITEM)):
    for j in range(len(RES)):
        for k in range(len(TIME)):
            obj_exp = obj_exp + prodqty[i][j][k] * int(adj_PCOST[i][j]) + storeqty[i][k] * int(adj_SCOST[i])
            
endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Make Objective:" + str(endDateTime-startDateTime))

Start : 2023-Feb-03 15:58:22.968331
End : 2023-Feb-03 15:58:28.947547
Make Objective:0:00:05.979216


In [14]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

solver.Minimize(obj_exp)
solver.Solve()

endDateTime = dt.now()
print("Solve Problem : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print(endDateTime-startDateTime)

Start : 2023-Feb-03 15:58:28.951632
Solve Problem : 2023-Feb-03 15:58:35.368370
0:00:06.416738


In [15]:
# for i in range(len(ITEM)):
#     for j in range(len(RES)):
#         for k in range(len(TIME)):
#             print ("ITEM %i, RES %i, WEEK %i " %(i+1,j+1,k+1) )
#             print ("Prod Qty",prodqty[i][j][k].solution_value())

In [16]:
# solver.Objective().Value()

In [17]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

arr_prod = np.zeros((prodqty_cnt,4))
itr_prod = 0
for i in range(len(ITEM)):
    for j in range(len(RES)):
        for k in range(len(TIME)):
            np.put(arr_prod,itr_prod*4  ,i+1)
            np.put(arr_prod,itr_prod*4+1,j+1)
            np.put(arr_prod,itr_prod*4+2,k+1)
            np.put(arr_prod,itr_prod*4+3,prodqty[i][j][k].solution_value())
            itr_prod = itr_prod+1 
 
df_prod= pd.DataFrame(arr_prod,columns = ['ITEM','RES','TIME','PROD'])

arr_store = np.zeros((storeqty_cnt,3))
itr_store = 0
for i in range(len(ITEM)):
    for k in range(len(TIME)):
        np.put(arr_store,itr_store*3  ,i+1)
        np.put(arr_store,itr_store*3+1,k+1)
        np.put(arr_store,itr_store*3+2,storeqty[i][k].solution_value())

df_store = pd.DataFrame(arr_store,columns = ['ITEM','TIME','STORE'])

df_prod.to_csv(path+'prodqty.csv',index=False)
df_store.to_csv(path+'storeqty.csv',index=False)
    
endDateTime = dt.now()
print("Write output : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print(endDateTime-startDateTime)

Start : 2023-Feb-03 15:58:35.381924
Write output : 2023-Feb-03 15:58:37.897561
0:00:02.515637
